In [ ]:
#%pip install seaborn
#%pip install pandas
#%pip install numpy
#%pip install zipfile
#%pip install matplotlib
#%pip install requests
import requests
import pandas as pd
import numpy as np
import os
import zipfile
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#List of countries and regions are here: https://unstats.un.org/unsd/methodology/m49/overview/ but website using
#javascript and not providing link for csv file directly
#Workaround:
#Using a github page to download a csv file
url = "https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/raw/master/all/all.csv"
filename = "countries.csv"

response = requests.get(url)

if response.status_code == 200:
    with open(filename, "wb") as file:
        file.write(response.content)
    print("CSV file downloaded successfully.")
else:
    print("Failed to download the CSV file.")

In [ ]:
df_countries=pd.read_csv("countries.csv")

In [ ]:
#To find a file of your interest: https://data.worldbank.org/indicator
#Clicking on a subset allows to get address to download csv-file

#These are addresses to obtain the data for GDP per capita, life expectancy and literacy rate
url_list=["https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.PP.CD?downloadformat=csv", \
          "https://api.worldbank.org/v2/en/indicator/SP.DYN.LE00.IN?downloadformat=csv",\
            "https://api.worldbank.org/v2/en/indicator/SE.ADT.LITR.ZS?downloadformat=csv"]

In [ ]:
#Also, giving names for the data to be obtained
data_list=["gdp_per_capita", "life_expectancy", "literacy"]

In [ ]:
#First, making folder to download the data from web, names based on data_list
for d in data_list:
    if not os.path.exists(d):
        os.makedirs(d)
    if not os.path.exists(d+"_unpacked"):
        os.makedirs(d+"_unpacked")

In [ ]:
#Here one by one, I dowload the data and put into a right folder
for d, u in zip(data_list, url_list):
    response = requests.get(u)
    if response.status_code == 200:
        with open(d+"/"+d+".zip", "wb") as file:
            file.write(response.content)
        print("Zip file downloaded successfully!")
    else:
        print("Failed to retrieve the zip file.")

In [ ]:
#Unpacking the data as well
for d in data_list:
    zip_file_path=d+"/"+d+".zip"
    destination_folder=d+"_unpacked"
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)
    print("Extraction completed!")


In [ ]:
#To remove excess rows in the csv file, an additional method:
def copy_file(source_file, destination_file):
    with open(source_file, 'r') as source:
        with open(destination_file, 'w') as destination:
            counter=0
            for line in source:
                if(counter>3):
                    destination.write(line)
                counter=counter+1

In [ ]:
#Removing extra rows from the csv file:
for d in data_list:
    file_list=os.listdir(d+"_unpacked")
    file_list.sort()
    source_file=d+"_unpacked/"+file_list[0]
    destination_file=d+".csv"
    copy_file(source_file, destination_file)

In [ ]:
#Using pandas, going to create the dataframe list:
df_list=[]
for d in data_list:
    temp_df=pd.read_csv(d+".csv")
    df_list.append(temp_df)

In [ ]:
#The last column is empty, must be removed
df_list[0].head(5)

In [ ]:
#Running the deletion
for n in range(len(df_list)):
    df_list[n]=df_list[n][df_list[n].columns[:-2]]

In [ ]:
#Checking
df_list[0].head()

In [ ]:
#Here I extract regions and sub-region from countries file and renaming a column
df_countries_small=df_countries[["name","alpha-3","region","sub-region"]]
df_countries_small = df_countries_small.rename(columns={'name': 'Country Name','alpha-3':'Country Code'})

In [ ]:
#Conducting merge on country names
#For gdp per capita
df_gdppcap=df_countries_small.merge(df_list[0], on='Country Code', how='inner')
#For life expectancy
df_lifeexp=df_countries_small.merge(df_list[1], on='Country Code', how='inner')
#For literacy
df_literacy=df_countries_small.merge(df_list[2], on='Country Code', how='inner')
##These are the dataframes one can work now!


In [ ]:
df_gdppcap.columns

In [ ]:
df_gdppcap = df_gdppcap.drop('Country Name_y', axis=1)
df_lifeexp = df_lifeexp.drop('Country Name_y', axis=1)
df_literacy = df_literacy.drop('Country Name_y', axis=1)

In [ ]:
df_gdppcap  = df_gdppcap.rename(columns={'Country Name_x': 'Country Name'})
df_lifeexp  = df_lifeexp.rename(columns={'Country Name_x': 'Country Name'})
df_literacy  = df_literacy.rename(columns={'Country Name_x': 'Country Name'})

In [ ]:
#Here saving dataframes as csv files for PowerBI
#These are time-series
if not os.path.exists('ForPowerBI'):
    os.makedirs('ForPowerBI')
df_gdppcap.to_csv('ForPowerBI/df_gdppcap.csv')
df_lifeexp.to_csv('ForPowerBI/df_lifeexp.csv')
df_literacy.to_csv('ForPowerBI/df_literacy.csv')

In [ ]:
#Also these same dataframes go to a separate folder: Dataframes_as_CSV
if not os.path.exists('Dataframes_as_CSV'):
    os.makedirs('Dataframes_as_CSV')
df_gdppcap.to_csv('Dataframes_as_CSV/df_gdppcap.csv')
df_lifeexp.to_csv('Dataframes_as_CSV/df_lifeexp.csv')
df_literacy.to_csv('Dataframes_as_CSV/df_literacy.csv')

In [ ]:
if not os.path.exists('Figures'):
    os.makedirs('Figures')

In [ ]:
#To get literacy values for year 2021, replacing NaN with np.nan
df_literacy=df_literacy.replace('NaN',np.nan)
#Filling np.nan with closest value to the left
df_literacy=df_literacy.ffill(axis=1)
df_literacy.head()

In [ ]:
#Getting values to year 2021
df_literacy_2021=df_literacy[['Country Name','region','sub-region','2021']]
df_literacy_2021.head()

In [ ]:
#One more change-removing these rows, where literacy could not be extrapolated
df_literacy_2021=df_literacy_2021[pd.to_numeric(df_literacy_2021['2021'], errors='coerce').notna()]
df_literacy_2021.head()

In [ ]:
#Getting gdp per capita for 2021
#To get literacy values for year 2021, replacing NaN with np.nan
df_gdppcap=df_gdppcap.replace('NaN',np.nan)
#Getting for year 2021
df_gdppcap_2021=df_gdppcap[['Country Name','2021']]
#Removing these rows, where np.nan
df_gdppcap_2021=df_gdppcap_2021.dropna()

In [ ]:
#Merging gdppcap_2021 with literacy_2021
df_gdppcap_literacy_2021=df_literacy_2021.merge(df_gdppcap_2021, on='Country Name', how='inner')
df_gdppcap_literacy_2021.head()


In [ ]:
df_gdppcap_literacy_2021 = df_gdppcap_literacy_2021.rename(columns={'2021_x': 'Literacy', '2021_y':'GDPPCAP'})
df_gdppcap_literacy_2021.head()

In [ ]:
sns.scatterplot(x="Literacy",
                    y="GDPPCAP",
                    data=df_gdppcap_literacy_2021, hue='region').set(title='GDP per Capita vs Literacy in 2021')
plt.legend(title='Region')
plt.savefig("Figures/Figure1.png")

In [ ]:
#Also that to csv file
df_gdppcap_literacy_2021.to_csv('Dataframes_as_CSV/df_gdppcap_literacy_2021.csv')

In [ ]:
#Here I want to extract only country and values for each year
#Extract years from column names
years=list(df_gdppcap.columns)[6:]
#A trick to add 'Country name' as a wanted column
Country=['Country Name']
#Extraction itself
df_gdppcap_country=df_gdppcap[Country+years]

In [ ]:
#Checking progress
df_gdppcap_country.head()

In [ ]:
df_gdppcap_country=df_gdppcap_country.dropna(subset=years, how='all')

In [ ]:
#Also making a dataframe for sub-regions and taking average for each year
df_gdppcap_subregion=df_gdppcap.groupby(['sub-region'])[years].mean().reset_index()


In [ ]:
#Such process would necessary for lineplot, melting
df_gdppcap_subregion.melt(id_vars=['sub-region'])

In [ ]:
#The dataframe has to be melted
plt.figure(figsize=(15, 15))

sns.lineplot(data=df_gdppcap_subregion.melt(id_vars=['sub-region']), 
             x='variable', y='value', hue='sub-region', palette='tab20',lw=5).set(title='GDP per capita in different regions')

plt.xticks(rotation=45)
plt.legend(title='Region')
plt.savefig("Figures/Figure2.png")
#No extra text as output, thus 'None' needed
None


In [ ]:
#Also that to csv
df_gdppcap_subregion.to_csv('Dataframes_as_CSV/df_gdppcap_subregion.csv')

In [ ]:
#Separate table for regions
gdp_per_capita_df = df_list[0]
regions_gdp_df = df_list[0].loc[gdp_per_capita_df['Country Name'].isin(["Africa Eastern and Southern",\
                                    "Africa Western and Central",\
                                    "Arab World",\
                                    "Central Europe and the Baltics",\
                                    "Caribbean small states",\
                                    "East Asia & Pacific (excluding high income)",\
                                    "East Asia & Pacific",\
                                    "Europe & Central Asia (excluding high income)",\
                                    "Europe & Central Asia",\
                                    "European Union",\
                                    "Latin America & Caribbean (excluding high income)",\
                                    "Latin America & Caribbean",\
                                    "Middle East & North Africa",\
                                    "Middle East & North Africa (excluding high income)",\
                                    "North America",\
                                    "Sub-Saharan Africa (excluding high income)",\
                                    "Sub-Saharan Africa"])]
regions_gdp_df.head()

In [ ]:
regions_gdp_df.columns

In [ ]:
regions_gdp_2021_df = regions_gdp_df[['Country Name', '2021']].reset_index(drop=True)
regions_gdp_2021_df.rename(columns = {'Country Name' : 'Region', '2021':'GDPPCAP'}, inplace = True)
regions_gdp_2021_df

In [ ]:
#Getting life expectancy for years, where we have values
life_expectancy_df = df_list[1]
life_expectancy_df2 = life_expectancy_df.drop(['Indicator Code', '1960', '1961',
       '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
       '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
       '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989'], axis=1)

In [ ]:
life_expectancy_2021_df=life_expectancy_df2[['Country Name', '2021']].reset_index(drop=True)
life_expectancy_2021_df.rename(columns = {'Country Name' : 'Region', '2021':'Life expectancy'}, inplace = True)
life_expectancy_2021_df

In [ ]:
#Joining tables together
Region_life_expectancy_GDPCAP_df=life_expectancy_2021_df.merge(regions_gdp_2021_df, on='Region', how='inner')
Region_life_expectancy_GDPCAP_df

In [ ]:
plt.figure(figsize=(10, 10))
sns.scatterplot(data=Region_life_expectancy_GDPCAP_df, x='Life expectancy', y='GDPPCAP', hue='Region').set_title('GDPPCAP vs Life expectancy')
plt.savefig("Figures/Figure3.png")

In [ ]:
#Also that as csv file:
Region_life_expectancy_GDPCAP_df.to_csv('Dataframes_as_CSV/Region_life_expectancy_GDPCAP_df.csv')

In [ ]:
#The same as above, but for individual countries
df_gdppcap=df_countries_small.merge(df_list[0], on='Country Name', how='inner')
#For life expectancy
df_lifeexp=df_countries_small.merge(df_list[1], on='Country Name', how='inner')

In [ ]:
df_gdppcap_2021=df_gdppcap[['Country Name','2021']].dropna()
df_gdppcap_2021 = df_gdppcap_2021.rename(columns={'2021': 'GDPPCAP'})
df_lifeexp_2021=df_lifeexp[['Country Name','region','2021']].dropna()
df_lifeexp_2021 = df_lifeexp_2021.rename(columns={'2021': 'Life expectancy'})


In [ ]:
df_lifeexp_gdpcap_2021=df_lifeexp_2021.merge(df_gdppcap_2021, on='Country Name', how='inner')
df_lifeexp_gdpcap_2021.head()

In [ ]:
plt.figure(figsize=(10, 10))
sns.scatterplot(data=df_lifeexp_gdpcap_2021, x='Life expectancy', y='GDPPCAP', hue='region').set_title('GDPPCAP vs Life expectancy')
#plt.legend(labels = ['Asia', 'Europe', 'Africa', 'Americas', 'Oceania'])
plt.legend(title='Region')
plt.savefig("Figures/Figure4.png")
None

In [ ]:
df_lifeexp_gdpcap_2021.to_csv('Dataframes_as_CSV/df_lifeexp_gdpcap_2021.csv')

In [ ]:
#Now going to extract data for Estonia
df_gdppcap_estonia=df_gdppcap.loc[df_gdppcap['Country Name'] == 'Estonia']

In [ ]:
df_gdppcap_estonia.head()

In [ ]:
#Extract years from column names, 1995-2021
years=list(df_gdppcap.columns)[-27:]


In [ ]:
estonia_gdppcap=df_gdppcap_estonia[years].values.flatten().tolist()            

In [ ]:
df_lifeexp_estonia=df_lifeexp.loc[df_lifeexp['Country Name'] == 'Estonia']

In [ ]:
estonia_lifeexp=df_lifeexp_estonia[years].values.flatten().tolist()

In [ ]:
est={'Year':years, 'GDPPCAP':estonia_gdppcap, 'Life expectancy':estonia_lifeexp}
df_estonia=pd.DataFrame(est)

In [ ]:
#df_estonia.melt(id_vars=['Year'])
df_estonia.head()

In [ ]:
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_title('GDPPCAP vs Life expectancy')
ax1.set_xlabel('Year')
ax1.set_ylabel('GDPPCAP', color=color)
ax1.plot(df_estonia['Year'], df_estonia['GDPPCAP'], color=color)
ax1.tick_params(axis='y', labelcolor=color)
plt.xticks(rotation=45)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Life expectancy', color=color) 
ax2.plot(df_estonia['Year'], df_estonia['Life expectancy'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.savefig("Figures/Figure5.png")


In [ ]:
#Also that to csv file
df_estonia.to_csv('Dataframes_as_CSV/df_estonia.csv')